# Integrations and Groups Calculator Notebook Demo
This is a quick demo for the Integrations and Groups Calculator (`ExoCTK.integrations_groups`). This demo can run with a `Python3` install, an up-to-date `ExoCTK` install, and the data file for the Integrations and Groups calculator (more on this later.) 

The Integrations and Groups Calculator was written as an exoplanet focused alternative to the JWST ETC or `pandeia`. It's power comes from :
1. Using pre-sampled data and interpolating for a speedy speedy speedy calculation. 
2. Having the power to provide you with the optimal configuration for your observation -- instead of just letting you guess and check configurations. 

This notebook has a few sections for clarity :
* [Imports and Setup](#imports)
* [Input Parameter Space](#input)
* [Building an Input Dictionary](#dict)
* [Running the Calculator](#calculate)
* [Exploring the Outputs ](#outputs)

<a id='imports'></a>
## Imports and Setup

Other than `ExoCTK` -- you'll need the pre-sampled data the Integrations and Groups calculator runs on in a `JSON` format. The file is available from the Data tab or bottom right corner of the [ExoCTK Website](https://exoctk.stsci.edu/). 

If you followed the regular installation you should have this under the environment variable `INTEGRATIONS_DIR`, and we'll just take it from your `path` -- otherwise you'll need to specify the path in this cell. 

Also -- **DON'T WORRY** if you get a big scary warning about the `matplotlib` backend. We need that in there to run server side when we host the website -- but it won't affect any of this demo of your general `ExoCTK` usage. If it's truly too hideous run this cell twice and warning should vanish.

In [2]:
# Imports
import json
import os

from exoctk.integrations_groups.integrations_groups import perform_calculation 

In [3]:
# Check if you have the environment variable
INTEGRATIONS_DIR = os.environ.get('INTEGRATIONS_DIR')
if INTEGRATIONS_DIR == '':
    INTEGRATIONS_DIR = '/path/to/your/local/copy/integrations_groups_data.json'

<a id='input'></a>
## Input Parameter Space 

We are often adding functionality and modes to this -- so instead of providing a static list of what models, instrument modes, etc., are presently feasible, here's a little demo on how to walk through the data file yourself and see what parameters are possible. 

In [4]:
# Open the file
with open(INTEGRATIONS_DIR) as f:
    parameter_space = json.load(f)
    
# Print the TA and Science observation modes
modes = [('science', 'sci_sat'), ('TA', 'ta_sat')]
for mode in modes:
    print('\n')
    print('Available modes for {} :'.format(mode[0]))
    print('------------------------')
    inss = list(parameter_space[mode[1]].keys())
    print('Instruments : {}'.format(inss))
    for ins in inss:
        filts = list(parameter_space[mode[1]][ins].keys())
        print('Filters for {} : {}'.format(ins, filts))
        subs = list(parameter_space[mode[1]][ins][filts[0]].keys())
        print('Subarrays for {} : {}'.format(ins, subs))

# Print the available Phoenix models
print('\n')
print('Phoenix model keys :')
print('---------------------')
print(list(parameter_space['ta_sat'][inss[-1]][filts[0]][subs[0]].keys()))

print('\n')
print('Magnitude sampling :')
print('--------------------')
print(parameter_space['mags'])



Available modes for science :
------------------------
Instruments : ['niriss', 'nircam', 'nirspec', 'miri']
Filters for niriss : ['soss']
Subarrays for niriss : ['substrip96', 'substrip256']
Filters for nircam : ['f444w', 'f277w', 'f322w2']
Subarrays for nircam : ['subgrism128', 'full', 'subgrism256', 'subgrism64']
Filters for nirspec : ['f070lp_g140h', 'f290lp_g395h', 'f170lp_g235m', 'f170lp_g235h', 'f070lp_g140m', 'f100lp_g140h', 'f100lp_g140m', 'f290lp_g395m']
Subarrays for nirspec : ['sub1024a', 'sub1024b', 'sub2048', 'sub512']
Filters for miri : ['lrs']
Subarrays for miri : ['slitlessprism']


Available modes for TA :
------------------------
Instruments : ['niriss', 'nircam', 'nirspec', 'miri']
Filters for niriss : ['f480m']
Subarrays for niriss : ['im', 'nrm']
Filters for nircam : ['f335m']
Subarrays for nircam : ['sub32tats']
Filters for nirspec : ['f110w', 'f140x', 'clear']
Subarrays for nirspec : ['sub2048', 'full', 'sub32']
Filters for miri : ['f1500w', 'f100w', 'f560w']


<a id='dict'></a>
## Building an Input Dictionary

Running the Integrations and Groups Calculator requires a dictionary of inputs. This section will go through an example input dictionary and what the limits on the parameters are.

In [5]:
# Initialize the dictionary
parameters = {}

# Source parameters
parameters['mag'] = 10 # 4.5 <= float <= 12.5
parameters['band'] = 'k' # only K band vega mag for now
parameters['mod'] = 'g2v' # Phoenix model per last section

# Observation specifics 
parameters['obs_time'] = 5 # positive float, in hours
parameters['n_group'] = 'optimize' # 'optimize', or positive integer

# Detector setup -- within the modes of the last section
parameters['ins'] = 'nircam'
# For science observation
parameters['filt'] = 'f444w'
parameters['subarray'] = 'subgrism256'
# And target acquisition
parameters['filt_ta'] = 'f335m'
parameters['subarray_ta'] = 'sub32tats'

# Saturation level
parameters['sat_mode'] = 'well' # 'well', for full well fraction, or 'counts' 
parameters['sat_max'] = .95 # < 1 for fullwell, and a positive integer always

# And feed in the data file
parameters['infile'] = INTEGRATIONS_DIR

<a id='calculate'></a>
## Running the Calculator 

Now, running the calculator is simple. We leaned on the `pandeia` function convention -- so feeding our inputs into `perform_calculation` returns a dictionary of input parameters (a stripped down `pandiea` scene) as well as the results of the calculation. (Note that `perform_calculation` updates the `parameters` dictionary -- so that object will be changed once you run the function.)

In [6]:
# Bookeeping for new/old parameters
inputs = list(parameters.keys())

# Perform the calculation 
results = perform_calculation(parameters)
for key in results:
    if key in inputs:
        if key == 'infile':
            # hackers
            print('The input of infile was REDACTED!')
        else:
            print('The input of {} was {}.'.format(key, results[key]))
    else:
        print('The result of {} was {}'.format(key, results[key]))


The input of mag was 10.
The input of band was k.
The input of mod was g2v.
The input of obs_time was 5.
The input of n_group was 147.
The input of ins was nircam.
The input of filt was f444w.
The input of subarray was subgrism256.
The input of filt_ta was f335m.
The input of subarray_ta was sub32tats.
The input of sat_mode was well.
The input of sat_max was 55195.0.
The input of infile was REDACTED!
The result of n_col was 256
The result of n_row was 256
The result of n_amp was 1
The result of n_reset was 1
The result of n_frame was 1
The result of n_skip was 0
The result of t_frame was 1.347
The result of t_int was 197.963
The result of t_ramp was 197.963
The result of n_int was 91
The result of t_exp was 5.004
The result of t_duration was 5.038
The result of obs_eff was 0.993
The result of ta_t_frame was 0.01496
The result of min_ta_groups was 33
The result of max_ta_groups was 3
The result of t_duration_ta_min was 0.50864
The result of t_duration_ta_max was 0.05984
The result of ma

<a id='outputs'></a>
## Exploring the Outputs

If you aren't quite familiar with the intricacies of a JWST observation, we'll unpack these results briefly. 

Every JWST observation has a number of groups and integrations. Groups are how many frames you can pack into an integration, and generally this is goverened by how quickly your target will saturate on the detector. With every integration there is overhead time added into the observation, and less time observing your actual transit. So, once the calculator has figured out the maximum possible groups before the detector is saturated, it will return that number of groups, how many integrations of that pattern it takes to fill up your whole transit time, and some additional helpful parameters like what's the maximum saturation you might reach during this observation, how long the actual scheme will take (since there will be a slightly different rounding when everything is added up), how efficient your observation is, etc. 

For target acquisition, the efficiency is less in question that the ability of the detector to hit the minimum SNR required. This provides a reccomendation, so you know the minimum and maxmimum possible groups you can use, and can make an informed decision. 

In [7]:
# So let's make a nice printed summary 
print('The total time for science + TA observation scheme is {}-{} hours.'.format(
    results['t_duration']+results['t_duration_ta_max'], results['t_duration']+results['t_duration_ta_min']))
print('You need {} groups and {} integrations for the science observation.'.format(
    results['n_group'], results['n_int']))
print('You need between {} and {} groups for target acquisition.'.format(
    results['max_ta_groups'], results['min_ta_groups']))
print('We estimate your science observation will reach at most {} counts -- how close were we to your cutoff of {}?'.format(
    results['max_sat_prediction'], results['sat_max']))
print('With this observation scheme {}% of the observation will be science data.'.format(results['obs_eff']*100))

The total time for science + TA observation scheme is 5.097840000000001-5.54664 hours.
You need 147 groups and 91 integrations for the science observation.
You need between 3 and 33 groups for target acquisition.
We estimate your science observation will reach at most 54944.595 counts -- how close were we to your cutoff of 55195.0?
With this observation scheme 99.3% of the observation will be science data.
